In [105]:
from dotenv import load_dotenv
load_dotenv()

True

In [106]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [107]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name = "messages")
    ]

)

chain = prompt | model

In [108]:
with_message_history = RunnableWithMessageHistory(
chain,
get_session_history,
input_messages_key = "messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Robert")], "language": "french"},
    config=config,
)

response.content


"Bonjour Robert ! Comment puis-je vous aider aujourd'hui ?"

In [109]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "french"},
    config=config,
)
response.content

'Votre nom est Robert. Comment puis-je vous aider, Robert ?'

In [110]:
#Managing  Conversaiton History
from langchain_core.messages import SystemMessage, trim_messages, AIMessage

trimmer = trim_messages(
    max_tokens = 100,
    strategy = "last",
    token_counter= model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),

]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [111]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | model)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content = "whats my name")],
        "language": "English",
    }
)
response.content

'your name is Bob.'

In [113]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what is the math question  I asked and my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked, "What\'s 2 + 2?" and your name is Bob.'